In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.activations import relu, linear
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.optimizers.legacy import RMSprop

In [2]:
Gitters = pd.read_csv('../data/hitters.csv')
Gitters.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [3]:
Gitters = pd.get_dummies(Gitters)
Gitters.drop(['League_N', 'Division_E', 'NewLeague_N'], axis=1, inplace=True)
Gitters.dropna(inplace=True)

In [4]:
X = Gitters.drop(['Salary'], axis=1)
y = Gitters.Salary

n, m = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [5]:
# Fit a linear regression model to hitters data
lfit = LinearRegression(fit_intercept=True)
lfit.fit(X_train, y_train)

# Compute the mean absolute error
np.mean(np.absolute(y_test-lfit.predict(X_test)))

267.07095551089486

In [6]:
# Fit lasso regression with cross-validation
lassocv = LassoCV(cv=10, random_state=0, max_iter=10000).fit(X_train, y_train)

np.mean(np.absolute(y_test-lassocv.predict(X_test)))

263.5245245666904

In [7]:
# Fit a neural network 
model = Sequential(
    [
        Dense(units=50, activation='relu', input_shape=(m, )),
        Dropout(0.4),
        Dense(units=1)
    ], name='hitters_model'
)

model.summary()

Model: "hitters_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                1000      
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 1051 (4.11 KB)
Trainable params: 1051 (4.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
# scaling input data

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [9]:
model.compile(
    loss=MeanAbsoluteError(),
    optimizer=RMSprop(),
    metrics=[MeanAbsoluteError()]
)

history = model.fit(X_train, y_train, epochs=1500, 
                    batch_size=32, validation_data=(X_test, y_test))

Epoch 1/1500
6/6 [==============================] - 0s 11ms/step - loss: 519.6552 - mean_absolute_error: 520.4284 - val_loss: 567.5153 - val_mean_absolute_error: 569.2108
Epoch 2/1500
6/6 [==============================] - 0s 2ms/step - loss: 519.2000 - mean_absolute_error: 522.6296 - val_loss: 567.1802 - val_mean_absolute_error: 568.8807
Epoch 3/1500
6/6 [==============================] - 0s 2ms/step - loss: 518.9643 - mean_absolute_error: 508.5414 - val_loss: 566.8793 - val_mean_absolute_error: 568.5844
Epoch 4/1500
6/6 [==============================] - 0s 2ms/step - loss: 518.6442 - mean_absolute_error: 526.5467 - val_loss: 566.6088 - val_mean_absolute_error: 568.3176
Epoch 5/1500
6/6 [==============================] - 0s 2ms/step - loss: 518.3080 - mean_absolute_error: 517.6979 - val_loss: 566.3425 - val_mean_absolute_error: 568.0544
Epoch 6/1500
6/6 [==============================] - 0s 2ms/step - loss: 518.1751 - mean_absolute_error: 521.3299 - val_loss: 566.0714 - val_mean_abso

Epoch 49/1500
6/6 [==============================] - 0s 2ms/step - loss: 501.4925 - mean_absolute_error: 514.1757 - val_loss: 548.2120 - val_mean_absolute_error: 550.1880
Epoch 50/1500
6/6 [==============================] - 0s 2ms/step - loss: 500.1652 - mean_absolute_error: 518.6221 - val_loss: 547.5806 - val_mean_absolute_error: 549.5673
Epoch 51/1500
6/6 [==============================] - 0s 2ms/step - loss: 499.7526 - mean_absolute_error: 501.1418 - val_loss: 546.9511 - val_mean_absolute_error: 548.9477
Epoch 52/1500
6/6 [==============================] - 0s 2ms/step - loss: 500.0051 - mean_absolute_error: 501.5081 - val_loss: 546.3305 - val_mean_absolute_error: 548.3376
Epoch 53/1500
6/6 [==============================] - 0s 2ms/step - loss: 498.9008 - mean_absolute_error: 494.6304 - val_loss: 545.6891 - val_mean_absolute_error: 547.7057
Epoch 54/1500
6/6 [==============================] - 0s 2ms/step - loss: 498.6578 - mean_absolute_error: 495.2751 - val_loss: 545.0254 - val_mean

Epoch 97/1500
6/6 [==============================] - 0s 2ms/step - loss: 468.5219 - mean_absolute_error: 487.7081 - val_loss: 513.5771 - val_mean_absolute_error: 515.6902
Epoch 98/1500
6/6 [==============================] - 0s 2ms/step - loss: 466.3405 - mean_absolute_error: 472.5594 - val_loss: 512.8799 - val_mean_absolute_error: 514.9772
Epoch 99/1500
6/6 [==============================] - 0s 2ms/step - loss: 466.3163 - mean_absolute_error: 465.0466 - val_loss: 512.1414 - val_mean_absolute_error: 514.2226
Epoch 100/1500
6/6 [==============================] - 0s 2ms/step - loss: 464.7522 - mean_absolute_error: 468.3051 - val_loss: 511.4741 - val_mean_absolute_error: 513.5447
Epoch 101/1500
6/6 [==============================] - 0s 2ms/step - loss: 464.2716 - mean_absolute_error: 466.8429 - val_loss: 510.8255 - val_mean_absolute_error: 512.8880
Epoch 102/1500
6/6 [==============================] - 0s 2ms/step - loss: 462.7392 - mean_absolute_error: 453.1034 - val_loss: 510.1053 - val_m

Epoch 145/1500
6/6 [==============================] - 0s 2ms/step - loss: 435.4398 - mean_absolute_error: 441.8123 - val_loss: 478.8557 - val_mean_absolute_error: 480.9382
Epoch 146/1500
6/6 [==============================] - 0s 2ms/step - loss: 431.8607 - mean_absolute_error: 421.6581 - val_loss: 478.0434 - val_mean_absolute_error: 480.1466
Epoch 147/1500
6/6 [==============================] - 0s 2ms/step - loss: 432.0673 - mean_absolute_error: 424.7602 - val_loss: 477.3190 - val_mean_absolute_error: 479.4344
Epoch 148/1500
6/6 [==============================] - 0s 2ms/step - loss: 429.6270 - mean_absolute_error: 439.2002 - val_loss: 476.5313 - val_mean_absolute_error: 478.6635
Epoch 149/1500
6/6 [==============================] - 0s 2ms/step - loss: 431.4869 - mean_absolute_error: 440.3650 - val_loss: 475.7479 - val_mean_absolute_error: 477.8936
Epoch 150/1500
6/6 [==============================] - 0s 2ms/step - loss: 430.8102 - mean_absolute_error: 432.7620 - val_loss: 475.0192 - va

Epoch 240/1500
6/6 [==============================] - 0s 2ms/step - loss: 366.6828 - mean_absolute_error: 367.0384 - val_loss: 396.8819 - val_mean_absolute_error: 400.9519
Epoch 241/1500
6/6 [==============================] - 0s 2ms/step - loss: 364.5699 - mean_absolute_error: 365.0761 - val_loss: 395.9036 - val_mean_absolute_error: 400.0006
Epoch 242/1500
6/6 [==============================] - 0s 2ms/step - loss: 365.6791 - mean_absolute_error: 363.9716 - val_loss: 394.9960 - val_mean_absolute_error: 399.1162
Epoch 243/1500
6/6 [==============================] - 0s 2ms/step - loss: 365.8928 - mean_absolute_error: 376.1041 - val_loss: 393.9740 - val_mean_absolute_error: 398.1260
Epoch 244/1500
6/6 [==============================] - 0s 2ms/step - loss: 364.0540 - mean_absolute_error: 357.5133 - val_loss: 393.0243 - val_mean_absolute_error: 397.1998
Epoch 245/1500
6/6 [==============================] - 0s 2ms/step - loss: 358.6131 - mean_absolute_error: 347.8510 - val_loss: 392.0674 - va

...

6/6 [==============================] - 0s 2ms/step - loss: 204.1197 - mean_absolute_error: 202.1180 - val_loss: 251.2498 - val_mean_absolute_error: 247.7367
Epoch 1095/1500
6/6 [==============================] - 0s 2ms/step - loss: 218.2322 - mean_absolute_error: 228.2884 - val_loss: 251.2497 - val_mean_absolute_error: 247.7332
Epoch 1096/1500
6/6 [==============================] - 0s 2ms/step - loss: 217.7109 - mean_absolute_error: 229.8920 - val_loss: 251.3188 - val_mean_absolute_error: 247.8259
Epoch 1097/1500
6/6 [==============================] - 0s 2ms/step - loss: 217.7205 - mean_absolute_error: 223.5997 - val_loss: 251.3144 - val_mean_absolute_error: 247.7988
Epoch 1098/1500
6/6 [==============================] - 0s 2ms/step - loss: 211.2231 - mean_absolute_error: 212.4057 - val_loss: 251.3009 - val_mean_absolute_error: 247.7764
Epoch 1099/1500
6/6 [==============================] - 0s 2ms/step - loss: 219.3751 - mean_absolute_error: 214.4037 - val_loss: 251.2945 - val_mean_abs

6/6 [==============================] - 0s 2ms/step - loss: 213.6584 - mean_absolute_error: 207.9017 - val_loss: 251.5488 - val_mean_absolute_error: 247.7612
Epoch 1189/1500
6/6 [==============================] - 0s 2ms/step - loss: 217.4194 - mean_absolute_error: 214.0912 - val_loss: 251.4587 - val_mean_absolute_error: 247.6693
Epoch 1190/1500
6/6 [==============================] - 0s 2ms/step - loss: 212.1108 - mean_absolute_error: 210.7501 - val_loss: 251.4741 - val_mean_absolute_error: 247.6931
Epoch 1191/1500
6/6 [==============================] - 0s 2ms/step - loss: 216.0729 - mean_absolute_error: 228.8247 - val_loss: 251.4385 - val_mean_absolute_error: 247.6786
Epoch 1192/1500
6/6 [==============================] - 0s 2ms/step - loss: 218.0476 - mean_absolute_error: 214.3678 - val_loss: 251.4559 - val_mean_absolute_error: 247.6929
Epoch 1193/1500
6/6 [==============================] - 0s 2ms/step - loss: 219.1625 - mean_absolute_error: 219.0466 - val_loss: 251.4371 - val_mean_abs

6/6 [==============================] - 0s 2ms/step - loss: 214.2982 - mean_absolute_error: 217.8355 - val_loss: 251.5704 - val_mean_absolute_error: 247.6413
Epoch 1283/1500
6/6 [==============================] - 0s 2ms/step - loss: 201.1940 - mean_absolute_error: 207.1277 - val_loss: 251.5408 - val_mean_absolute_error: 247.6001
Epoch 1284/1500
6/6 [==============================] - 0s 2ms/step - loss: 208.5747 - mean_absolute_error: 211.8113 - val_loss: 251.5757 - val_mean_absolute_error: 247.6440
Epoch 1285/1500
6/6 [==============================] - 0s 2ms/step - loss: 199.2219 - mean_absolute_error: 195.8340 - val_loss: 251.5883 - val_mean_absolute_error: 247.6485
Epoch 1286/1500
6/6 [==============================] - 0s 2ms/step - loss: 209.3075 - mean_absolute_error: 205.7875 - val_loss: 251.6297 - val_mean_absolute_error: 247.7121
Epoch 1287/1500
6/6 [==============================] - 0s 2ms/step - loss: 206.5244 - mean_absolute_error: 198.6595 - val_loss: 251.6024 - val_mean_abs

6/6 [==============================] - 0s 2ms/step - loss: 204.7713 - mean_absolute_error: 201.3224 - val_loss: 251.5305 - val_mean_absolute_error: 247.4683
Epoch 1377/1500
6/6 [==============================] - 0s 2ms/step - loss: 202.3441 - mean_absolute_error: 201.0358 - val_loss: 251.5498 - val_mean_absolute_error: 247.5004
Epoch 1378/1500
6/6 [==============================] - 0s 2ms/step - loss: 216.5992 - mean_absolute_error: 213.5078 - val_loss: 251.5715 - val_mean_absolute_error: 247.5557
Epoch 1379/1500
6/6 [==============================] - 0s 3ms/step - loss: 214.1845 - mean_absolute_error: 218.1068 - val_loss: 251.5591 - val_mean_absolute_error: 247.5394
Epoch 1380/1500
6/6 [==============================] - 0s 2ms/step - loss: 211.3265 - mean_absolute_error: 210.8395 - val_loss: 251.6732 - val_mean_absolute_error: 247.6510
Epoch 1381/1500
6/6 [==============================] - 0s 2ms/step - loss: 206.2346 - mean_absolute_error: 207.7589 - val_loss: 251.6596 - val_mean_abs

6/6 [==============================] - 0s 2ms/step - loss: 200.8181 - mean_absolute_error: 201.7040 - val_loss: 251.8481 - val_mean_absolute_error: 247.8326
Epoch 1471/1500
6/6 [==============================] - 0s 2ms/step - loss: 205.3658 - mean_absolute_error: 204.9124 - val_loss: 251.8238 - val_mean_absolute_error: 247.7962
Epoch 1472/1500
6/6 [==============================] - 0s 2ms/step - loss: 200.9456 - mean_absolute_error: 209.6336 - val_loss: 251.9003 - val_mean_absolute_error: 247.8964
Epoch 1473/1500
6/6 [==============================] - 0s 2ms/step - loss: 201.1636 - mean_absolute_error: 211.3310 - val_loss: 251.8835 - val_mean_absolute_error: 247.8676
Epoch 1474/1500
6/6 [==============================] - 0s 2ms/step - loss: 202.9696 - mean_absolute_error: 205.0457 - val_loss: 251.8422 - val_mean_absolute_error: 247.8154
Epoch 1475/1500
6/6 [==============================] - 0s 2ms/step - loss: 209.9963 - mean_absolute_error: 201.0281 - val_loss: 251.8490 - val_mean_abs

In [10]:
# Evaluate model on test data
model.evaluate(X_test, y_test)

3/3 [==============================] - 0s 761us/step - loss: 251.8200 - mean_absolute_error: 247.7599


[251.82003784179688, 247.7599334716797]

| Model                      | Mean Absolute Error |
|----------------------------|---------------------|
| Linear Regression          | 267.0710            |
| Lasso (with Cross-validation) | 263.52              |
| Neural Network (1 Hidden layer + Dropout Layer) | 247.7599            |